In [7]:
!python -m pip install --upgrade pip
%pip install openpyxl

import pandas as pd
import numpy as np
%pip install nltk vaderSentiment # Install libraries for sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
excel_file = pd.ExcelFile("Stock Price & Tweets.xlsx") 

# Load the 2 excel sheets to 2 different data frame
df1 = excel_file.parse('Numeric Data')
df2 = excel_file.parse('Tweets')

display(df1.head())
display(df2.head())

,day,open_price,high_price,low_price,moving_average_5_day,moving_average_10_day,moving_average_50_day,moving_average_200_day,volume,next_day_close_price
0,1,44.55,44.83,44.32,44.36,43.82,40.23,42.25,1500700,44.83
1,2,44.89,45.12,44.17,44.37,44.06,40.32,42.27,2391800,45.03
2,3,45.04,45.35,44.84,44.57,44.21,40.42,42.27,1723400,44.97
3,4,45.02,45.15,44.70,44.65,44.40,40.50,42.28,1490500,44.91
4,5,44.75,45.04,44.65,44.76,44.58,40.59,42.28,1349500,45.31


,day,tweet
0,1,"#Dan ($Dan) Doubles Down on Healthy, Eco-Frien..."
1,1,RT @DvdndDiplomats: Bert's X Always Buy stocks...
2,1,$Dan Alert From our Stock News Alerts App
3,1,X NEW Stocks at #FusionIQ with Master Scores >...
4,1,"#AmazonPrime creates a captive audience, so ""b..."


In [11]:
# Initialize sentiment intensity analyzer: Create an instance of the VADER Sentiment Intensity Analyzer.
analyzer = SentimentIntensityAnalyzer()

# Apply the analyzer to the 'tweet' column of df2 to get sentiment scores.
df2['tweet'] = df2['tweet'].astype(str)
df2['sentiment_scores'] = df2['tweet'].apply(analyzer.polarity_scores)
df2['compound_score'] = df2['sentiment_scores'].apply(lambda x: x['compound'])

display(df2.head())

,day,tweet,sentiment_scores,compound_score
0,1,"#Dan ($Dan) Doubles Down on Healthy, Eco-Frien...","{'neg': 0.0, 'neu': 0.722, 'pos': 0.278, 'comp...",0.4019
1,1,RT @DvdndDiplomats: Bert's X Always Buy stocks...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,1,$Dan Alert From our Stock News Alerts App,"{'neg': 0.0, 'neu': 0.761, 'pos': 0.239, 'comp...",0.2960
3,1,X NEW Stocks at #FusionIQ with Master Scores >...,"{'neg': 0.0, 'neu': 0.881, 'pos': 0.119, 'comp...",0.3182
4,1,"#AmazonPrime creates a captive audience, so ""b...","{'neg': 0.0, 'neu': 0.877, 'pos': 0.123, 'comp...",0.2732


In [12]:
# Calculate the average compound score for each day
average_compound_score_per_day = df2.groupby('day')['compound_score'].mean()
print("Average compound score per day:")
display(average_compound_score_per_day.head())

Average compound score per day:


day
1    0.118089
2    0.122978
3    0.182958
4    0.188611
5    0.162906
Name: compound_score, dtype: float64

In [13]:
# Merge df1 with the average compound score per day on the 'day' column
df1 = pd.merge(df1, average_compound_score_per_day, on='day', how='left')

display(df1.head())

,day,open_price,high_price,low_price,moving_average_5_day,moving_average_10_day,moving_average_50_day,moving_average_200_day,volume,next_day_close_price,compound_score
0,1,44.55,44.83,44.32,44.36,43.82,40.23,42.25,1500700,44.83,0.118089
1,2,44.89,45.12,44.17,44.37,44.06,40.32,42.27,2391800,45.03,0.122978
2,3,45.04,45.35,44.84,44.57,44.21,40.42,42.27,1723400,44.97,0.182958
3,4,45.02,45.15,44.70,44.65,44.40,40.50,42.28,1490500,44.91,0.188611
4,5,44.75,45.04,44.65,44.76,44.58,40.59,42.28,1349500,45.31,0.162906


In [14]:
# Create predict data frame
df_predict = df1[pd.isnull(df1.next_day_close_price) == True].copy()

# Create train data frame
df_main = df1[pd.isnull(df1.next_day_close_price) == False].copy()

In [15]:
#split data into training and testing sets
df_train, df_test = train_test_split(df_main, test_size=0.3, shuffle=True, random_state=42)

display(df_train.shape)
display(df_test.shape)

(700, 11)

(300, 11)

In [16]:
# Create the feature vector by dropping 'next_day_close_price' from df_main
train_features = df_train.drop(columns=['next_day_close_price','day'])
test_features = df_test.drop(columns=['next_day_close_price','day'])
predict_features = df_predict.drop(columns=['next_day_close_price','day'])

In [17]:
#get the training and testing labels (polarity labels)
train_labels = df_train.next_day_close_price#.to_list()
test_labels = df_test.next_day_close_price#.to_list()

In [18]:
#define a linear regression classifier
model_1 = LinearRegression()

#train the linear regression model using the training data
model_1.fit(train_features, train_labels)

# Make predictions on the training data
train_predictions = model_1.predict(train_features)

# Evaluate the model on the training data
train_mse = mean_squared_error(train_labels, train_predictions)
train_r2 = r2_score(train_labels, train_predictions)

print(f'Training Mean Squared Error: {train_mse:.3f}')
print(f'Training R-squared: {train_r2:.3f}')

# Make predictions on the testing data
test_predictions = model_1.predict(test_features)

# Evaluate the model on the testing data
test_mse = mean_squared_error(test_labels, test_predictions)
test_r2 = r2_score(test_labels, test_predictions)

print(f'Testing Mean Squared Error: {test_mse:.3f}')
print(f'Testing R-squared: {test_r2:.3f}')

Training Mean Squared Error: 0.595
Training R-squared: 0.994
Testing Mean Squared Error: 0.679
Testing R-squared: 0.994


In [19]:
# Make predictions on the predict data set
predictions = model_1.predict(predict_features)

In [21]:
# Create a DataFrame with 'day' and 'predictions'
output_df = pd.DataFrame({'day': df_predict['day'], 'next_day_close_price': predictions})

display(output_df.head())

,day,next_day_close_price
1000,1001,55.925160
1001,1002,58.822778
1002,1003,58.576092
1003,1004,58.038790
1004,1005,58.617253
